In [1]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

In [2]:
%%capture
!pip install wandb -qU
# Log in to your W&B account

import wandb
wandb.login(key="509ed54310c255777d71f02d1e36bf5c8e49a443")
# 빈 칸이 뜨면 API key 입력하기

In [3]:
# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")

In [4]:
config_data = {
    "general": {
        "data_path": "../data/",  # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
        "model_name": "digit82/kobart-summarization",  # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
        "output_dir": "./"  # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
    },
    "tokenizer": {
        "encoder_max_len": 512,  # 인코더의 최대 길이를 설정합니다. # 512
        "decoder_max_len": 200,  # 디코더의 최대 길이를 설정합니다. # 200
        "bos_token": f"{tokenizer.bos_token}",  # 시작 토큰을 설정합니다.
        "eos_token": f"{tokenizer.eos_token}",  # 종료 토큰을 설정합니다.
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
        # "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#']
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#']
    },
    "training": {
        "overwrite_output_dir": True,  # 출력 디렉토리를 덮어쓸지 여부를 설정합니다.
        "num_train_epochs": 20,  # 학습 에폭 수를 설정합니다.
        "learning_rate": 1e-5,  # 학습률을 설정합니다.
        "per_device_train_batch_size": 50,  # 단일 장치당 학습 배치 크기를 설정합니다.
        "per_device_eval_batch_size": 32,  # 단일 장치당 평가 배치 크기를 설정합니다.
        "warmup_ratio": 0.1,  # warmup 비율을 설정합니다.
        "weight_decay": 0.01,  # 가중치 감쇠를 설정합니다.
        "lr_scheduler_type": 'cosine',  # 학습률 스케줄러 타입을 설정합니다.
        "optim": 'adamw_torch',  # 옵티마이저를 설정합니다.
        "gradient_accumulation_steps": 1,  # 그래디언트 누적 스텝을 설정합니다.
        "evaluation_strategy": 'epoch',  # 평가 전략을 설정합니다.
        "save_strategy": 'epoch',  # 저장 전략을 설정합니다.
        "save_total_limit": 5,  # 총 저장 제한을 설정합니다.
        "fp16": True,  # FP16 학습을 사용할지 여부를 설정합니다.
        "load_best_model_at_end": True,  # 최고의 모델을 학습 종료 시 불러올지 여부를 설정합니다.
        "seed": 42,  # 시드 값을 설정합니다.
        "logging_dir": "./logs",  # 로그 디렉토리를 설정합니다.
        "logging_strategy": "epoch",  # 로깅 전략을 설정합니다.
        "predict_with_generate": True,  # 생성을 통한 예측을 사용할지 여부를 설정합니다.
        "generation_max_length": 100,  # 생성의 최대 길이를 설정합니다.
        "do_train": True,  # 학습을 수행할지 여부를 설정합니다.
        "do_eval": True,  # 평가를 수행할지 여부를 설정합니다.
        "early_stopping_patience": 3,  # 조기 종료의 인내 값 설정합니다. # !!!!! 3
        "early_stopping_threshold": 0.001,  # 조기 종료 임계값을 설정합니다. # !!!!! 0.001
        "report_to": "wandb"  # wandb에 보고할지 여부를 설정합니다.
    },
    # wandb 홈페이지에 가입하여 얻은 정보를 기반으로 작성합니다.
    "wandb": {
        "entity": "kangs7777",  # wandb의 entity를 설정합니다.
        "project": "nlp-seunghyun",  # wandb의 프로젝트를 설정합니다.
        "name": "kobert(1_2)"  # 표시할 이름을 설정합니다. # !!!!!
    },
    "inference": {
        "ckt_path": "model ckt path",  # 사전 학습이 진행된 모델의 checkpoint를 저장할 경로를 설정합니다.
        "result_path": "./prediction/",  # 결과를 저장할 경로를 설정합니다.
        "no_repeat_ngram_size": 2,  # 반복되는 n-gram 크기를 설정합니다.
        "early_stopping": True,  # 조기 종료를 사용할지 여부를 설정합니다.
        "generate_max_length": 200,  # 생성의 최대 길이를 설정합니다.
        "num_beams": 4,  # 빔 검색을 위한 빔 크기를 설정합니다.
        "batch_size": 32,  # 배치 크기를 설정합니다.
        # 정확한 모델 평가를 위해 제거할 불필요한 생성 토큰들을 정의합니다.
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    }
}


In [5]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
config_path = "./config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

In [6]:
# 저장된 config 파일을 불러옵니다.
config_path = "./config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 불러온 config 파일의 전체 내용을 확인합니다.
pprint(loaded_config)

{'general': {'data_path': '../data/',
             'model_name': 'digit82/kobart-summarization',
             'output_dir': './'},
 'inference': {'batch_size': 32,
               'ckt_path': 'model ckt path',
               'early_stopping': True,
               'generate_max_length': 200,
               'no_repeat_ngram_size': 2,
               'num_beams': 4,
               'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>'],
               'result_path': './prediction/'},
 'tokenizer': {'bos_token': '<s>',
               'decoder_max_len': 200,
               'encoder_max_len': 512,
               'eos_token': '</s>',
               'special_tokens': ['#Person1#',
                                  '#Person2#',
                                  '#Person3#',
                                  '#Person4#',
                                  '#Person5#',
                                  '#Person6#',
                                  '#Person7#']},
 'training': {'do_eval': True,
          

In [7]:
# 데이터 전처리를 하기 위해 tokenization 과정에서 필요한 정보들을 확인합니다.
loaded_config['tokenizer']

{'bos_token': '<s>',
 'decoder_max_len': 200,
 'encoder_max_len': 512,
 'eos_token': '</s>',
 'special_tokens': ['#Person1#',
  '#Person2#',
  '#Person3#',
  '#Person4#',
  '#Person5#',
  '#Person6#',
  '#Person7#']}

In [8]:
# 모델이 훈련 시 적용될 매개변수를 확인합니다.
loaded_config['training']

{'do_eval': True,
 'do_train': True,
 'early_stopping_patience': 3,
 'early_stopping_threshold': 0.001,
 'evaluation_strategy': 'epoch',
 'fp16': True,
 'generation_max_length': 100,
 'gradient_accumulation_steps': 1,
 'learning_rate': 1e-05,
 'load_best_model_at_end': True,
 'logging_dir': './logs',
 'logging_strategy': 'epoch',
 'lr_scheduler_type': 'cosine',
 'num_train_epochs': 20,
 'optim': 'adamw_torch',
 'overwrite_output_dir': True,
 'per_device_eval_batch_size': 32,
 'per_device_train_batch_size': 50,
 'predict_with_generate': True,
 'report_to': 'wandb',
 'save_strategy': 'epoch',
 'save_total_limit': 5,
 'seed': 42,
 'warmup_ratio': 0.1,
 'weight_decay': 0.01}

In [9]:
# 모델이 최종 결과를 출력하기 위한 매개변수 정보를 확인합니다.
loaded_config['inference']

{'batch_size': 32,
 'ckt_path': 'model ckt path',
 'early_stopping': True,
 'generate_max_length': 200,
 'no_repeat_ngram_size': 2,
 'num_beams': 4,
 'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>'],
 'result_path': './prediction/'}

In [10]:
# config에 저장된 데이터 경로를 통해 train과 validation data를 불러옵니다.
data_path = loaded_config['general']['data_path']

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(data_path,'train.csv'))
train_df.tail()

,fname,dialogue,summary,topic
12452,train_12455,#Person1#: 실례합니다. 맨체스터 출신의 그린 씨이신가요?\n#Person2...,탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다....,누군가를 태우다
12453,train_12456,#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 ...,#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12454,train_12457,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
12455,train_12458,#Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일 있어?\n#Person2...,#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직
12456,train_12459,"#Person1#: 엄마, 다음 토요일에 이 삼촌네 가족을 방문하기 위해 비행기를 ...",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 ...,짐 싸기


In [11]:
# validation data의 구조와 내용을 확인합니다.
val_df = pd.read_csv(os.path.join(data_path,'dev.csv'))
val_df.tail()

,fname,dialogue,summary,topic
494,dev_495,#Person1#: 이제 새해가 되어서 새로운 시작을 하려고 결심했어. \r\n#P...,#Person1#은 새해에 금연을 하고 커밍아웃하기로 결정했습니다. #Person2...,새해
495,dev_496,"#Person1#: 너, 조랑 결혼했지? \r\n#Person2#: 조? 무슨 말인...",#Person1#은 #Person2#가 조와 결혼했다고 생각했다. #Person2#...,사랑에 빠지다
496,dev_497,"#Person1#: 무엇을 도와드릴까요, 부인?\r\n#Person2#: 몇 주 동...",#Person2#의 차에서 이상한 소리가 납니다. #Person1#는 브레이크를 교...,소음
497,dev_498,"#Person1#: 안녕하세요, 아마존 고객 서비스입니다. 무엇을 도와드릴까요?\n...",#Person2#님이 아마존 고객 서비스에 전화하여 아마존에서 받은 책에 한 페이지...,빠진 페이지
498,dev_499,#Person1#: 여름이 다 되어간다는 게 믿기지 않아.\r\n#Person2#:...,#Person2#는 #Person1#에게 여름 휴가 동안 파티를 도와주는 회사에서 ...,여름 휴가


In [12]:
# # 대화의 재구성
# import random

# def augment_dialogue_by_reordering(dialogue):
#     # 대화를 문장 단위로 분리
#     sentences = dialogue.split('\n')
#     # 문장 순서를 랜덤하게 재배열
#     random.shuffle(sentences)
#     # 재배열된 문장을 다시 합쳐 새로운 대화 생성
#     new_dialogue = '\n'.join(sentences)
#     return new_dialogue

# print(train_df.shape)

# augmented_dialogues_by_reordering = train_df['dialogue'].apply(augment_dialogue_by_reordering)
# augmented_df_by_reordering = train_df.copy()
# augmented_df_by_reordering['dialogue'] = augmented_dialogues_by_reordering
# train_df = pd.concat([train_df, augmented_df_by_reordering], ignore_index=True)

# print(train_df.shape)

In [13]:
# 백-트랜슬레이션
from transformers import MarianMTModel, MarianTokenizer

def back_translate(text, src_lang="ko", mid_lang="en"):
    # 소스 언어에서 중간 언어로 번역
    tokenizer_src_to_mid = MarianTokenizer.from_pretrained(f'Helsinki-NLP/opus-mt-tc-big-{src_lang}-{mid_lang}')
    model_src_to_mid = MarianMTModel.from_pretrained(f'Helsinki-NLP/opus-mt-tc-big-{src_lang}-{mid_lang}')
    
    translated_to_mid = model_src_to_mid.generate(**tokenizer_src_to_mid(text, return_tensors="pt", padding=True))
    text_mid = tokenizer_src_to_mid.decode(translated_to_mid[0], skip_special_tokens=True)
    
    # 중간 언어에서 소스 언어로 번역
    tokenizer_mid_to_src = MarianTokenizer.from_pretrained(f'Helsinki-NLP/opus-mt-tc-big-{mid_lang}-{src_lang}')
    model_mid_to_src = MarianMTModel.from_pretrained(f'Helsinki-NLP/opus-mt-tc-big-{mid_lang}-{src_lang}')
    
    translated_back_to_src = model_mid_to_src.generate(**tokenizer_mid_to_src(text_mid, return_tensors="pt", padding=True))
    text_back = tokenizer_mid_to_src.decode(translated_back_to_src[0], skip_special_tokens=True)
    
    return text_back

print(train_df.shape)

augmented_dialogues_by_back_translate = train_df['dialogue'].apply(lambda x: back_translate(x, src_lang="ko", mid_lang="en"))
augmented_df_by_back_translate = train_df.copy()
augmented_df_by_back_translate['dialogue'] = augmented_dialogues_by_back_translate
train_df = pd.concat([train_df, augmented_df_by_back_translate], ignore_index=True)

print(train_df.shape)

(12457, 4)


In [ ]:
import re
from konlpy.tag import Okt

# 불필요한 텍스트 제거
def remove_noise(dialogue):
    dialogue = re.sub(r'\[.*?\]|\{.*?\}', '', dialogue)  # 대괄호와 중괄호 안의 텍스트 제거
    dialogue = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9.,!?#:\n ]', '', dialogue)  # 특수 문자 제거, #, :, \n은 유지
    return dialogue

# 문장 단위 분리 및 대화자 태그 정규화
def split_and_normalize_speaker(dialogue):
    #dialogue = dialogue.replace("#Person1#", "<인물1>").replace("#Person2#", "<인물2>")  # 대화자 태그 정규화
    sentences = dialogue.split('\n')  # 문장 단위로 분리
    return '\n'.join(sentences)  # 분리된 문장들을 \n으로 다시 결합

# 전처리 함수 적용
def preprocess_dataframe_0(df):
    df['dialogue'] = df['dialogue'].apply(remove_noise).apply(split_and_normalize_speaker)
    return df

# 스탑워드 리스트 (예시)
stopwords = ['은', '는', '이', '가', '하', '아', '것', '들', '의', '있', '되', '수', '보', '주', '등', '한']

# 불필요한 텍스트 제거, 문장 단위 분리, 대화자 태그 정규화, 스탑워드 제거
def preprocess_dialogue(dialogue):
    # 대화자 태그 정규화 및 불필요한 텍스트 제거
    dialogue = re.sub(r'\[.*?\]|\{.*?\}', '', dialogue)
    dialogue = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9.,!?#:\n ]', '', dialogue)
    
    # 문장 단위 분리 및 스탑워드 제거
    okt = Okt()
    processed_dialogue = []
    for line in dialogue.split('\n'):
        if line.strip() == "": continue  # 빈 줄은 건너뜀
        speaker_tag, _, sentence = line.partition(':')
        tokens = okt.morphs(sentence.strip())
        meaningful_words = [word for word in tokens if word not in stopwords]
        processed_sentence = ' '.join(meaningful_words)
        processed_dialogue.append(f"{speaker_tag}:{processed_sentence}")
    
    return '\n'.join(processed_dialogue)

# 데이터프레임에 전처리 함수 적용
def preprocess_dataframe_1(df):
    df['dialogue'] = df['dialogue'].apply(preprocess_dialogue)
    return df


In [ ]:
##### !!!!!
#train_df = preprocess_dataframe_0(train_df)
train_df = preprocess_dataframe_1(train_df)

#val_df = preprocess_dataframe_0(val_df)
val_df = preprocess_dataframe_1(val_df)

display(train_df), display(val_df)

In [ ]:
# # 대화의 재구성
# import random

# def augment_dialogue_by_reordering(dialogue):
#     # 대화를 문장 단위로 분리
#     sentences = dialogue.split('\n')
#     # 문장 순서를 랜덤하게 재배열
#     random.shuffle(sentences)
#     # 재배열된 문장을 다시 합쳐 새로운 대화 생성
#     new_dialogue = '\n'.join(sentences)
#     return new_dialogue

# print(train_df.shape)

# augmented_dialogues_by_reordering = train_df['dialogue'].apply(augment_dialogue_by_reordering)
# augmented_df_by_reordering = train_df.copy()
# augmented_df_by_reordering['dialogue'] = augmented_dialogues_by_reordering
# train_df = pd.concat([train_df, augmented_df_by_reordering], ignore_index=True)

# print(train_df.shape)

In [ ]:
# # 백-트랜슬레이션
# from transformers import MarianMTModel, MarianTokenizer

# def back_translate(text, src_lang="ko", mid_lang="en"):
#     # 소스 언어에서 중간 언어로 번역
#     tokenizer_src_to_mid = MarianTokenizer.from_pretrained(f'Helsinki-NLP/opus-mt-{src_lang}-{mid_lang}')
#     model_src_to_mid = MarianMTModel.from_pretrained(f'Helsinki-NLP/opus-mt-{src_lang}-{mid_lang}')
    
#     translated_to_mid = model_src_to_mid.generate(**tokenizer_src_to_mid(text, return_tensors="pt", padding=True))
#     text_mid = tokenizer_src_to_mid.decode(translated_to_mid[0], skip_special_tokens=True)
    
#     # 중간 언어에서 소스 언어로 번역
#     tokenizer_mid_to_src = MarianTokenizer.from_pretrained(f'Helsinki-NLP/opus-mt-{mid_lang}-{src_lang}')
#     model_mid_to_src = MarianMTModel.from_pretrained(f'Helsinki-NLP/opus-mt-{mid_lang}-{src_lang}')
    
#     translated_back_to_src = model_mid_to_src.generate(**tokenizer_mid_to_src(text_mid, return_tensors="pt", padding=True))
#     text_back = tokenizer_mid_to_src.decode(translated_back_to_src[0], skip_special_tokens=True)
    
#     return text_back

# print(train_df.shape)

# augmented_dialogues_by_back_translate = train_df['dialogue'].apply(lambda x: back_translate(x, src_lang="ko", mid_lang="en"))
# augmented_df_by_back_translate = train_df.copy()
# augmented_df_by_back_translate['dialogue'] = augmented_dialogues_by_back_translate
# train_df = pd.concat([train_df, augmented_df_by_back_translate], ignore_index=True)

# print(train_df.shape)

In [ ]:
# 데이터 전처리를 위한 클래스로, 데이터셋을 데이터프레임으로 변환하고 인코더와 디코더의 입력을 생성합니다.
class Preprocess:
    def __init__(self,
            bos_token: str,
            eos_token: str,
        ) -> None:

        self.bos_token = bos_token
        self.eos_token = eos_token

    @staticmethod
    # 실험에 필요한 컬럼을 가져옵니다.
    def make_set_as_df(file_path, is_train = True):
        if is_train:
            df = pd.read_csv(file_path)
            train_df = df[['fname','dialogue','summary']]
            return train_df
        else:
            df = pd.read_csv(file_path)
            test_df = df[['fname','dialogue']]
            return test_df

    # BART 모델의 입력, 출력 형태를 맞추기 위해 전처리를 진행합니다.
    def make_input(self, dataset,is_test = False):
        if is_test:
            encoder_input = dataset['dialogue']
            decoder_input = [self.bos_token] * len(dataset['dialogue'])
            return encoder_input.tolist(), list(decoder_input)
        else:
            encoder_input = dataset['dialogue']
            decoder_input = dataset['summary'].apply(lambda x : self.bos_token + str(x)) # Ground truth를 디코더의 input으로 사용하여 학습합니다.
            decoder_output = dataset['summary'].apply(lambda x : str(x) + self.eos_token)
            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()


In [ ]:
# Train에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        return item

    def __len__(self):
        return self.len

# Validation에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForVal(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        return item

    def __len__(self):
        return self.len

# Test에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForInference(Dataset):
    def __init__(self, encoder_input, test_id, len):
        self.encoder_input = encoder_input
        self.test_id = test_id
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        item['ID'] = self.test_id[idx]
        return item

    def __len__(self):
        return self.len


In [ ]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_train_dataset(config, preprocessor, data_path, tokenizer):
    train_file_path = os.path.join(data_path,'train.csv')
    val_file_path = os.path.join(data_path,'dev.csv')

    # train, validation에 대해 각각 데이터프레임을 구축합니다.
    train_data = preprocessor.make_set_as_df(train_file_path)
    val_data = preprocessor.make_set_as_df(val_file_path)

    print('-'*150)
    print(f'train_data:\n {train_data["dialogue"][0]}')
    print(f'train_label:\n {train_data["summary"][0]}')

    print('-'*150)
    print(f'val_data:\n {val_data["dialogue"][0]}')
    print(f'val_label:\n {val_data["summary"][0]}')

    encoder_input_train , decoder_input_train, decoder_output_train = preprocessor.make_input(train_data)
    encoder_input_val , decoder_input_val, decoder_output_val = preprocessor.make_input(val_data)
    print('-'*10, 'Load data complete', '-'*10,)

    tokenized_encoder_inputs = tokenizer(encoder_input_train, return_tensors="pt", padding=True,
                            add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_inputs = tokenizer(decoder_input_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_ouputs = tokenizer(decoder_output_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    train_inputs_dataset = DatasetForTrain(tokenized_encoder_inputs, tokenized_decoder_inputs, tokenized_decoder_ouputs,len(encoder_input_train))

    val_tokenized_encoder_inputs = tokenizer(encoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_inputs = tokenizer(decoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_ouputs = tokenizer(decoder_output_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    val_inputs_dataset = DatasetForVal(val_tokenized_encoder_inputs, val_tokenized_decoder_inputs, val_tokenized_decoder_ouputs,len(encoder_input_val))

    print('-'*10, 'Make dataset complete', '-'*10,)
    return train_inputs_dataset, val_inputs_dataset

In [ ]:
def compute_metrics(config, tokenizer, pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거합니다.
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token, " ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token, " ") for sentence in replaced_labels]
        
    print('-'*150)
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[1]}")
    print(f"GOLD: {replaced_labels[1]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[2]}")
    print(f"GOLD: {replaced_labels[2]}")
    
    # ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels)

    # 데이터 프레임을 생성하기 위한 준비
    data = {'GOLD': replaced_labels, 'PRED': replaced_predictions}

    # ROUGE 점수를 데이터프레임에 추가합니다.
    for metric in ['rouge-1', 'rouge-2', 'rouge-l']:
        for key in ['f', 'p', 'r']:
            values = [score[metric][key] for score in results]
            data[f'{metric}_{key.upper()}'] = values

    # 데이터프레임 생성
    df = pd.DataFrame(data)

    # 저장할 파일 경로
    file_path = 'validation/kobert(1_2)/kobert.csv' # !!!!!

    # 기존 파일이 있는지 확인
    if os.path.exists(file_path):
        # 파일이 이미 존재하면 새로운 파일로 저장
        i = 1
        while os.path.exists(f"{os.path.splitext(file_path)[0]}_{i}.csv"):
            i += 1
        file_path = f"{os.path.splitext(file_path)[0]}_{i}.csv"

    # 데이터프레임을 CSV 파일로 저장
    df.to_csv(file_path, index=False)
    
    # 최종적인 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)
    
    # ROUGE 점수 중 F-1 score를 통해 평가합니다.
    result = {key: value["f"] for key, value in results.items()}
    
    return result

In [ ]:
# 학습을 위한 trainer 클래스와 매개변수를 정의합니다.
def load_trainer_for_train(config,generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset):
    print('-'*10, 'Make training arguments', '-'*10,)
    # set training args
    training_args = Seq2SeqTrainingArguments(
                output_dir=config['general']['output_dir'], # model output directory
                overwrite_output_dir=config['training']['overwrite_output_dir'],
                num_train_epochs=config['training']['num_train_epochs'],  # total number of training epochs
                learning_rate=config['training']['learning_rate'], # learning_rate
                per_device_train_batch_size=config['training']['per_device_train_batch_size'], # batch size per device during training
                per_device_eval_batch_size=config['training']['per_device_eval_batch_size'],# batch size for evaluation
                warmup_ratio=config['training']['warmup_ratio'],  # number of warmup steps for learning rate scheduler
                weight_decay=config['training']['weight_decay'],  # strength of weight decay
                lr_scheduler_type=config['training']['lr_scheduler_type'],
                optim =config['training']['optim'],
                gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
                evaluation_strategy=config['training']['evaluation_strategy'], # evaluation strategy to adopt during training
                save_strategy =config['training']['save_strategy'],
                save_total_limit=config['training']['save_total_limit'], # number of total save model.
                fp16=config['training']['fp16'],
                load_best_model_at_end=config['training']['load_best_model_at_end'], # 최종적으로 가장 높은 점수 저장
                seed=config['training']['seed'],
                logging_dir=config['training']['logging_dir'], # directory for storing logs
                logging_strategy=config['training']['logging_strategy'],
                predict_with_generate=config['training']['predict_with_generate'], #To use BLEU or ROUGE score
                generation_max_length=config['training']['generation_max_length'],
                do_train=config['training']['do_train'],
                do_eval=config['training']['do_eval'],
                report_to=config['training']['report_to'] # (선택) wandb를 사용할 때 설정합니다.
            )

    # (선택) 모델의 학습 과정을 추적하는 wandb를 사용하기 위해 초기화 해줍니다.
    wandb.init(
        entity=config['wandb']['entity'],
        project=config['wandb']['project'],
        name=config['wandb']['name'],
    )

    # (선택) 모델 checkpoint를 wandb에 저장하도록 환경 변수를 설정합니다.
    os.environ["WANDB_LOG_MODEL"]="true"
    os.environ["WANDB_WATCH"]="false"

    # Validation loss가 더 이상 개선되지 않을 때 학습을 중단시키는 EarlyStopping 기능을 사용합니다.
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],
        early_stopping_threshold=config['training']['early_stopping_threshold']
    )
    print('-'*10, 'Make training arguments complete', '-'*10,)
    print('-'*10, 'Make trainer', '-'*10,)

    # Trainer 클래스를 정의합니다.
    trainer = Seq2SeqTrainer(
        model=generate_model, # 사용자가 사전 학습하기 위해 사용할 모델을 입력합니다.
        args=training_args,
        train_dataset=train_inputs_dataset,
        eval_dataset=val_inputs_dataset,
        compute_metrics = lambda pred: compute_metrics(config,tokenizer, pred),
        callbacks = [MyCallback]
    )
    print('-'*10, 'Make trainer complete', '-'*10,)

    return trainer

In [ ]:
# 학습을 위한 tokenizer와 사전 학습된 모델을 불러옵니다.
def load_tokenizer_and_model_for_train(config,device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)
    print('-'*10, f'Model Name : {config["general"]["model_name"]}', '-'*10,)
    model_name = config['general']['model_name']
    bart_config = BartConfig().from_pretrained(model_name)
    generate_model = BartForConditionalGeneration.from_pretrained(config['general']['model_name'],config=bart_config)

    special_tokens_dict={'additional_special_tokens':config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model.resize_token_embeddings(len(tokenizer)) # 사전에 special token을 추가했으므로 재구성 해줍니다.
    generate_model.to(device)
    print(generate_model.config)

    print('-'*10, 'Load tokenizer & model complete', '-'*10,)
    return generate_model , tokenizer

In [ ]:
def main(config):
    # 사용할 device를 정의합니다.
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    # 사용할 모델과 tokenizer를 불러옵니다.
    generate_model , tokenizer = load_tokenizer_and_model_for_train(config,device)
    print('-'*10,"tokenizer special tokens : ",tokenizer.special_tokens_map,'-'*10)

    # 학습에 사용할 데이터셋을 불러옵니다.
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token']) # decoder_start_token: str, eos_token: str
    data_path = config['general']['data_path']
    train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config,preprocessor, data_path, tokenizer)

    # Trainer 클래스를 불러옵니다.
    trainer = load_trainer_for_train(config, generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset)
    trainer.train()   # 모델 학습을 시작합니다.

    # (선택) 모델 학습이 완료된 후 wandb를 종료합니다.
    wandb.finish()

In [ ]:
if __name__ == "__main__":
    main(loaded_config)

In [ ]:
loaded_config['inference']['ckt_path'] = "/data/ephemeral/home/code/checkpoint-2250"

In [ ]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_test_dataset(config,preprocessor, tokenizer):

    test_file_path = os.path.join(config['general']['data_path'],'test.csv')

    test_data = preprocessor.make_set_as_df(test_file_path,is_train=False)
    
    ############# !!!!!
    #test_data = preprocess_dataframe_0(test_data)
    test_data = preprocess_dataframe_1(test_data)
    #test_data['processed_dialogue'] = test_data['dialogue'].apply(preprocess_dialogue_for_each_person_with_newline)
    #test_data = pd.DataFrame({'fname':test_data.fname, 'dialogue':test_data.processed_dialogue})
    
    
    test_id = test_data['fname']

    print('-'*150)
    print(f'test_data:\n{test_data["dialogue"][0]}')
    print('-'*150)

    encoder_input_test , decoder_input_test = preprocessor.make_input(test_data,is_test=True)
    print('-'*10, 'Load data complete', '-'*10,)

    test_tokenized_encoder_inputs = tokenizer(encoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False,)
    test_tokenized_decoder_inputs = tokenizer(decoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False,)

    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('-'*10, 'Make dataset complete', '-'*10,)

    return test_data, test_encoder_inputs_dataset

In [ ]:
# 추론을 위한 tokenizer와 학습시킨 모델을 불러옵니다.
def load_tokenizer_and_model_for_test(config,device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)

    model_name = config['general']['model_name']
    ckt_path = config['inference']['ckt_path']
    print('-'*10, f'Model Name : {model_name}', '-'*10,)
    tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="right", max_new_tokens=1000)
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model = BartForConditionalGeneration.from_pretrained(ckt_path)
    generate_model.resize_token_embeddings(len(tokenizer))
    generate_model.to(device)
    print('-'*10, 'Load tokenizer & model complete', '-'*10,)

    return generate_model , tokenizer

In [ ]:
# 학습된 모델이 생성한 요약문의 출력 결과를 보여줍니다.
def inference(config):
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    generate_model , tokenizer = load_tokenizer_and_model_for_test(config,device)

    data_path = config['general']['data_path']
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'])

    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config, preprocessor, tokenizer)
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []
    with torch.no_grad():
        for item in tqdm(dataloader):
            text_ids.extend(item['ID'])
            generated_ids = generate_model.generate(input_ids=item['input_ids'].to('cuda:0'),
                            no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                            early_stopping=config['inference']['early_stopping'],
                            max_length=config['inference']['generate_max_length'],
                            num_beams=config['inference']['num_beams'],
                        )
            for ids in generated_ids:
                result = tokenizer.decode(ids)
                summary.append(result)

    # 정확한 평가를 위하여 노이즈에 해당되는 스페셜 토큰을 제거합니다.
    remove_tokens = config['inference']['remove_tokens']
    preprocessed_summary = summary.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    output = pd.DataFrame(
        {
            "fname": test_data['fname'],
            "summary" : preprocessed_summary,
        }
    )
    result_path = config['inference']['result_path']
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    output.to_csv(os.path.join(result_path, "kobert(1_2)_output.csv"), index=False) # !!!!!!

    return output

In [ ]:
# 학습된 모델의 test를 진행합니다.
if __name__ == "__main__":
    output = inference(loaded_config)

In [ ]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.